## Packages

In [ ]:
#Packages to solve DE and math
import numpy as np
from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d, UnivariateSpline
import scipy.fftpack
from statsmodels.nonparametric.kernel_regression import KernelReg

#Packages to plot and animate
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.animation import PillowWriter  
from collections import deque

#Packages from imripy
from imripy import merger_system as ms
from imripy import inspiral
from imripy import waveform
from imripy import detector
from imripy import halo
from imripy import cosmo

#miscellaneous Packages
import time

## Orbit

In [ ]:
def initial_conditions(sp, a0, e0, phi0 = 0.):
    """
    Calculate the initial conditions for a Keplerian orbit with parameters a, e
    """
    r0 = a0 * (1. - e0**2) / (1. + e0 * np.cos(phi0))
    dphi0 = np.sqrt(sp.m_total() * a0 * (1.-e0**2)) / r0**2
    dr0 = a0* (1. - e0**2) / (1 + e0 * np.cos(phi0))**2 * e0 *np.sin(phi0) * dphi0
    return [r0, phi0, dr0, dphi0, sp.m2]

def dm2dt(sp, r, v):
    sigma = (np.pi * sp.m2**2. / v**2.) * (8. * (1. - v**2.))**3 / (4. * (1. - 4. * v**2. + (1. + 8. * v**2.)**(1./2.)) * (3. - (1. + 8. * v**2.)**(1./2.))**2.)
    return sigma * sp.halo.density(r) * v
    

def evolve(sp, y0, t_end, t_start=0., gwEmission=True, dynamicalFriction=True, accretion=True, 
                           accretionRecoil=True, coulombLog=3., nSteps=1000, acc=1e-9):   
    """
    Evolve the system of differential equations from t_start to t_end with initial conditions y0
    """
    t_scale = t_end
    r_scale = y0[0]
    m_scale = y0[4]
    
    y0scaled = y0[:]
    y0scaled[0] /= r_scale; y0scaled[2] /= r_scale; y0scaled[4]/= m_scale
    
    t_eval = np.linspace(t_start, t_end, nSteps)/t_scale
    
    def dy_dt(t, y, sp):
        r, phi, dr, dphi, m2 = y
        r *= r_scale; dr *= r_scale; m2 *= m_scale; sp.m2 = m2
        
        v = np.sqrt(dr**2 + r**2 * dphi**2)
        
        gwEmissionTerm = (8./15.* sp.m_reduced() * sp.m_total()**2 / r**4 / v**2
                               * (12. *r**2 * dphi**2 + dr**2 )) if gwEmission else 0.

        dm2_dt = dm2dt(sp, r, v) if accretion else 0.
        mass_change = sp.m1/sp.m2/sp.m_total() * dm2_dt    # this is dmu/mu
        accretionRecoilTerm = dm2_dt / sp.m_reduced()   if accretionRecoil else 0.
         
        dynamicalFrictionTerm = 4.*np.pi * sp.m2**2/sp.m_reduced() * sp.halo.density(r) * coulombLog /v**3  if dynamicalFriction else 0.
           
        ddr = (  r * dphi**2  - sp.m_total()  / r**2
                  - mass_change *dr
                  - accretionRecoilTerm   * dr
                  - gwEmissionTerm * dr
                  - dynamicalFrictionTerm * dr
                  ) 
        
        ddphi= (-2.* dr * dphi / r
                  - mass_change * dphi
                  - accretionRecoilTerm * dphi
                  - gwEmissionTerm  *dphi
                  - dynamicalFrictionTerm * dphi
               ) 
        
        dy_dt = np.array([dr/r_scale, dphi, ddr/r_scale, ddphi, dm2_dt/m_scale]) * t_scale
        #print(y, dy_dt)
        return  dy_dt 
    
    rt_start = time.perf_counter()
    sol=solve_ivp(dy_dt, [t_start/t_scale, t_end/t_scale], y0scaled, method='RK45', args=(sp, ),
                          dense_output=True, atol=1e-12, rtol=acc, t_eval=t_eval)
    rt_elapsed = time.perf_counter() - rt_start
    
    sol.t *= t_scale
    sol.y[0] *= r_scale
    sol.y[2] *= r_scale
    sol.y[4] *= m_scale
    print(sol.message)
    print("elapsed time ", rt_elapsed)
    return sol

In [ ]:
def plotOrbit(sp, sol, ax_polar, label="", **args):
    r = sol.y[0]; phi = sol.y[1]
    trace, = ax_polar.plot(phi, r/sp.r_isco(), linestyle='--', lw=1, label=label, **args)

In [ ]:
# Set basic parameters of the system
m1= 1e3* ms.solar_mass_to_pc
m2= 1. * ms.solar_mass_to_pc
D = 1e5 

sp_0 = ms.SystemProp(m1, m2, halo.ConstHalo(0.), D)

In [ ]:
# Set initial parameters of the inspiral
a0 = 100. * sp_0.r_isco()
e0 = 0.
y0 = initial_conditions(sp_0, a0, e0)
print(y0)
F0 = np.sqrt(sp_0.m_total()/a0**3)/ 2./np.pi
nOrbits = 1000
t_end = nOrbits/ F0

In [ ]:
# Evolve with and without gw loss
sol_0 = evolve(sp_0, y0, t_end, gwEmission=False, dynamicalFriction=False, accretion=False, nSteps=20*nOrbits)
sol_0gw = evolve(sp_0, y0, t_end, gwEmission=True, dynamicalFriction=False, accretion=False, nSteps=20*nOrbits, acc=1e-10)

In [ ]:
# Plot orbit
fig = plt.figure(figsize=(9,9))
ax = fig.add_subplot(projection='polar')
plotOrbit(sp_0, sol_0, ax, 'no GW')
plotOrbit(sp_0, sol_0gw, ax, 'with GW', alpha=0.5)
fig.legend();

In [ ]:
def getOrbitalParameters(sp, sol, smooth_e=False):
    """
    Calculate a, e as Keplerian paramters from the evolution
    """
    r = sol.y[0]; phi = sol.y[1]; dr = sol.y[2]; dphi = sol.y[3]
    
    v = np.sqrt(dr**2 + r**2 * dphi**2)
    a = sp.m_total()/ np.abs( v**2 - 2*sp.m_total()/r)
    e2 = 1. + ( v**2 - 2*sp.m_total()/r ) * r**4 * dphi**2 / sp.m_total()**2
    e = np.sqrt(np.clip(e2, 0., None))
    #e = np.sqrt(e2)
    
    if smooth_e:
        n = 5000
        split_e = np.array_split(e, len(e)//n)
        split_t = np.array_split(sol.t, len(e)//n)
        for i in range(len(split_e)):
            t_s = time.perf_counter()
            kr = KernelReg(split_e[i], split_t[i], 'c')
            split_e[i] = kr.fit(split_t[i])[0]
            t_e = time.perf_counter()
            print(f"Kernel Regression step {i} took {t_e-t_s}s: {(i+1) * n}/{len(e)} ")
        e = np.concatenate(split_e, axis=0)
    
    sol.v = v; sol.a = a; sol.e = e
    return sol
    
def plotOrbitalParameters(sp, sol, ax_a, ax_e, label="", timescale = 1.):
    #if not hasattr(sol, 'a'):
    sol = getOrbitalParameters(sp, sol)
    l, = ax_a.plot(sol.t/timescale, sol.a/sp.r_isco(), label=label)
    ax_e.plot(sol.t/timescale, sol.e, color=l.get_c())
    #ax_e.plot(t, sol.y[1], linestyle='--', color=l.get_c())
    #ax_e.plot(t, np.cos(sol.y[1]), linestyle='-.', color=l.get_c())
    return sol

In [ ]:
# Plot keplerian elements
fig, (ax_a, ax_e) = plt.subplots(2,1, sharex='all', figsize=(10, 8))
fig.subplots_adjust(hspace=0.)
sol_0 = plotOrbitalParameters(sp_0, sol_0, ax_a, ax_e, label='no GW', timescale=1/F0)
sol_0gw = plotOrbitalParameters(sp_0, sol_0gw, ax_a, ax_e, label='with GW', timescale=1/F0)
ax_a.grid(); ax_a.set_ylabel('a')
ax_e.grid(); ax_e.set_ylabel('e'); ax_e.set_xlabel('t')
ax_a.legend()

In [ ]:
sol_0gw = getOrbitalParameters(sp_0, sol_0gw, smooth_e=True) # smooth e so that np.gradient returns sensible results
sol_0 = getOrbitalParameters(sp_0, sol_0, smooth_e=True) # smooth e so that np.gradient returns sensible results

In [ ]:
# Compare to 4.116, 4.117    # mix from original and new
def da_dt(sp, a, e):
    return (-64./5. *sp.m_reduced()*sp.m_total()**2 / a**3  / (1.-e**2)**(7./2.) 
                    *(1. + 73./24.*e**2 + 37./96.*e**4))
def de_dt(sp, a, e):
    return (-304./15. *sp.m_reduced()*sp.m_total()**2 / a**4  * e/ (1.-e**2)**(5./2.) 
                *(1. + 121./304.*e**2 ) )
plt.plot(sol_0gw.t, da_dt(sp_0, sol_0gw.a, sol_0gw.e), label='a, analytic')
plt.plot(sol_0gw.t, np.gradient(sol_0gw.a, sol_0gw.t), linestyle='--', label="a, code")
plt.plot(sol_0gw.t, de_dt(sp_0, sol_0gw.a, sol_0gw.e), label='e, analytic')
plt.plot(sol_0gw.t, np.gradient(sol_0gw.e, sol_0gw.t), linestyle='--', label="e, code")
plt.legend(); plt.grid()
plt.yscale('symlog')

In [ ]:
# Compare to 4.25 of Maggiore
if e0 == 0.:
    t_coal = 5./256. * a0**4 / sp_0.m_total()**2 / sp_0.m_reduced()
    plt.plot(sol_0gw.t, a0 * ((t_coal - sol_0gw.t)/t_coal)**(1./4.), label='analytic')
    plt.plot(sol_0gw.t, sol_0gw.y[0], label='code', linestyle='--')
    #plt.plot(sol_0.t, sol_0.y[0], label='code, no gw')
    plt.legend(); plt.grid()
    plt.show()

In [ ]:
if e0 > 0.:   # Compare to 4.127
    def g(e):
        return e**(12./19.)/(1.-e**2) * (1. + 121./304.*e**2)**(870./2299.)
    plt.plot(sol_0gw.e, sol_0gw.a[0] * g(sol_0gw.e)/ g(sol_0gw.e[0]), label='analytic')
    plt.plot(sol_0gw.e, sol_0gw.a, linestyle='--', label="code")
    plt.legend(); plt.grid()
    plt.show()

### Gravitational Wave Signal

In [ ]:
def strain(sp, sol):
    r = sol.y[0]; phi = sol.y[1];
    t = sol.t
    #Observer parameters
    mu = sp.m_reduced()
    theta_o = sp.inclination_angle
    phi_o = sp.pericenter_angle

    #Rotating body parameters
    x = r*np.cos(phi)
    y = r*np.sin(phi)
    z = np.zeros(np.shape(x))
    
    Q=[ [mu*x*x, mu*x*y, mu*x*z],
           [mu*y*x, mu*y*y, mu*y*z],
           [mu*z*x, mu*z*y, mu*z*z] ]
    
    def Mdt(Q):
        return np.array([ [np.gradient(Q[0][0], t), np.gradient(Q[0][1], t), np.gradient(Q[0][2], t)], 
                            [np.gradient(Q[1][0], t),np.gradient(Q[1][1], t),np.gradient(Q[1][2], t)],
                            [np.gradient(Q[2][0], t),np.gradient(Q[2][1], t),np.gradient(Q[2][2], t)] ])
    dQdt=Mdt(Q)
    d2Qd2t=Mdt(dQdt)
    
    h_plus =  1./sp.D * ( d2Qd2t[0][0] * (np.cos(phi_o)**2 - np.sin(phi_o)**2 * np.cos(theta_o)**2) 
                         + d2Qd2t[1][1] * (np.sin(phi_o)**2 - np.cos(phi_o)**2 * np.cos(theta_o)**2) 
                         - d2Qd2t[2][2] * np.sin(theta_o)**2 
                         - d2Qd2t[0][1] * np.sin(2*phi_o) * (1. + np.cos(theta_o)**2)
                         + d2Qd2t[0][2] * np.sin(phi_o) * np.sin(2*theta_o) 
                         + d2Qd2t[1][2] * np.cos(phi_o) * np.sin(2*theta_o) )      
    h_cross = 1./sp.D * ( (d2Qd2t[0][0]-d2Qd2t[1][1]) * np.sin(2*phi_o)*np.cos(theta_o)
                         + 2. * d2Qd2t[0][1] * np.cos(2*phi_o)*np.cos(theta_o) 
                         - 2. * d2Qd2t[0][2] * np.cos(phi_o)*np.sin(theta_o) 
                         + 2. * d2Qd2t[1][2] * np.sin(theta_o)*np.sin(phi_o) )
    
    sol.h_plus = h_plus; sol.h_cross = h_cross
    return sol

def strainFFT(sp, sol, f_bin):
    N = len(sol.t)
    T = sol.t[1] - sol.t[0]
    
    h_plus_fft = scipy.fftpack.fft(sol.h_plus)
    h_cross_fft = scipy.fftpack.fft(sol.h_cross)
    xf = scipy.fftpack.fftfreq(N, T)[:N//2]
    
    h_plus_fft = h_plus_fft[np.where((xf > f_bin[0]) & (xf < f_bin[1]))]
    h_cross_fft = h_cross_fft[np.where((xf > f_bin[0]) & (xf < f_bin[1]))]
    xf = xf[np.where((xf > f_bin[0]) & (xf < f_bin[1]))]
    
    sol.f = xf; sol.h_plus_fft = h_plus_fft; sol.h_cross_fft = h_cross_fft
    return sol


In [ ]:
def plot_strain(sp, sol, ax_h_plus, ax_f=None, label="", plot_h_cross=False):
    l, = ax_h_plus.plot(sol.t/ms.year_to_pc, sol.h_plus, label=label)
    if plot_h_cross:
        ax_h_cross.plot(sol.t/ms.year_to_pc, sol.h_cross, color=l.get_c(), linestyle='--')
    if ax_f is None:
        return
    ax_f.plot(sol.f/ms.hz_to_invpc, 2.*sol.f*np.abs(sol.h_plus_fft), color=l.get_c())


In [ ]:
# Calculate GW signal
sol_0 = strain(sp_0, sol_0)
sol_0 = strainFFT(sp_0, sol_0, [1e-4*ms.hz_to_invpc, 1e-1*ms.hz_to_invpc])

In [ ]:
# Plot GW signal
fig, (ax_h_plus, ax_f) = plt.subplots(2, 1, figsize=(10,12))
plot_strain(sp_0, sol_0, ax_h_plus, ax_f=ax_f, label="no GW")

ax_h_plus.set_xlabel("t / yr"); ax_h_plus.set_ylabel("$h_+$")
ax_h_plus.grid(); ax_h_plus.legend(); ax_h_plus.set_xlim(left=0., right=10./F0/ms.year_to_pc)
ax_f.set_xlabel("f / Hz"); ax_f.set_ylabel("characteristic strain"); ax_f.grid(); 
print("orbital freq: ", F0/ms.hz_to_invpc)

### Compare to imripy

In [ ]:
# Define different dark matter models
m1= 1e3* ms.solar_mass_to_pc
m2= 1. * ms.solar_mass_to_pc
D = 1e5 

rho_spike = 226. * ms.solar_mass_to_pc
alpha_spike = 7./3.
r_spike = 0.54
sp_spike = ms.SystemProp(m1, m2, halo.Spike(rho_spike, r_spike, alpha_spike), D)

In [ ]:
# Initial conditions
inspiral.Classic.ln_Lambda = 3.
inspiral.Classic.dmPhaseSpaceFraction=1.

a0 = 30. * sp_spike.r_isco()
e0 = 0.1
y0 = initial_conditions(sp_spike, a0, e0)
print(y0)
F0 = np.sqrt(sp_spike.m_total()/a0**3)/ 2./np.pi
nOrbits = 2000
t_end = nOrbits/ F0

In [ ]:
def compareModels(sp, sol, ev, ax_a, ax_e=None, label=""):
    l1, = ax_a.plot(sol.t, sol.a, label=label + ",GR")
    l2, = ax_a.plot(ev.t, ev.a, label=label + ",imripy", linestyle='--')
    #l3, = ax_a.plot(ev.t, np.abs(interp1d(sol.t, sol.a, kind='cubic', bounds_error=False, fill_value=(0.,0.))(ev.t) - ev.a),
    #              label='$\Delta$')
    if not ax_e is None:
        ax_e.plot(sol.t, sol.e, color=l1.get_c())
        if isinstance(ev.e, np.ndarray):
            ax_e.plot(ev.t, ev.e, linestyle='--', color=l2.get_c())
        else:
            ax_e.plot(ev.t, np.zeros(np.shape(ev.t)), linestyle='--', color=l2.get_c())
        #ax_e.plot(ev.t, np.abs(interp1d(sol.t, sol.e, kind='cubic', bounds_error=False, fill_value=(0.,0.))(ev.t) - ev.e),
        #          label='$\Delta$', color=l3.get_c())

In [ ]:
# Evolution of both this code and imripy with GW loss and dynamical friction
sol_spike = evolve(sp_spike, y0, t_end, nSteps=20*nOrbits, accretion=False, acc=1e-10)
sol_spike = getOrbitalParameters(sp_spike, sol_spike)
ev_spike = inspiral.Classic.Evolve(sp_spike, a0, e_0=e0, t_fin = t_end)

In [ ]:
# Plot difference
fig, (ax_a, ax_e) = plt.subplots(2, 1, figsize=(10, 12))
compareModels(sp_spike, sol_spike, ev_spike, ax_a, ax_e=ax_e if e0 > 0. else None, label="spike")

ax_a.grid(); ax_a.legend()
ax_a.set_yscale('log')
ax_e.grid(); 

In [ ]:
# Evolution of both this code and imripy with just GWloss
sol_gwOnly = evolve(sp_spike, y0, t_end, nSteps=10*nOrbits, dynamicalFriction=False, accretion=False, acc=1e-12)
sol_gwOnly = getOrbitalParameters(sp_spike, sol_gwOnly)
ev_gwOnly = inspiral.Classic.Evolve(sp_spike, a0, e_0=e0, t_fin = t_end, 
                            opt=inspiral.Classic.EvolutionOptions(gwEmissionLoss=True, accretion=False, dynamicalFrictionLoss=False))

In [ ]:
fig, (ax_a, ax_e) = plt.subplots(2, 1, figsize=(10, 12))
compareModels(sp_spike, sol_gwOnly, ev_gwOnly, ax_a, ax_e=ax_e, label="gwOnly")

ax_a.grid(); ax_a.legend()
ax_a.set_yscale('log')
ax_e.grid(); 

In [ ]:
# Evolution of both this code and imripy with just dynamical friction Loss
sol_dfOnly = evolve(sp_spike, y0, t_end, nSteps=10*nOrbits, gwEmission=False, accretion=False, acc=1e-12)
sol_dfOnly = getOrbitalParameters(sp_spike, sol_dfOnly)
ev_dfOnly = inspiral.Classic.Evolve(sp_spike, a0, e_0=e0, t_fin = t_end, opt=inspiral.Classic.EvolutionOptions(dynamicalFrictionLoss=True, accretion=False, gwEmissionLoss=False))

In [ ]:
fig, (ax_a, ax_e) = plt.subplots(2, 1, figsize=(10, 12))
compareModels(sp_spike, sol_dfOnly, ev_dfOnly, ax_a, ax_e=ax_e, label="dfOnly")

ax_a.grid(); ax_a.legend()
ax_a.set_yscale('log')
ax_e.grid(); 

In [ ]:
# Evolution of both this code and imripy with just accretion effects
sol_accOnly = evolve(sp_spike, y0, t_end, nSteps=10*nOrbits, gwEmission=False, dynamicalFriction=False, accretion=True, accretionRecoil=True, acc=1e-10)
sol_accOnly = getOrbitalParameters(sp_spike, sol_accOnly)
sp_spike.m2 = m2
ev_accOnly = inspiral.Classic.Evolve(sp_spike, a0, e_0=e0, t_fin = t_end, 
                                    opt=inspiral.Classic.EvolutionOptions(accretion=True, accretionForceLoss=True, accretionRecoilLoss=True, verbose=1, 
                                                                          gwEmissionLoss=False, dynamicalFrictionLoss=False, accuracy=1e-10))

In [ ]:
fig, (ax_a, ax_e, ax_m) = plt.subplots(3, 1, figsize=(10, 12))
compareModels(sp_spike, sol_accOnly, ev_accOnly, ax_a, ax_e=ax_e, label="accOnly")
ax_m.plot(sol_accOnly.t, sol_accOnly.y[4]/m2 - 1.)
ax_m.plot(ev_accOnly.t, ev_accOnly.m2/m2 - 1., linestyle='--')

ax_a.grid(); ax_a.legend()
ax_a.set_yscale('log')
ax_e.grid(); ax_m.grid()

In [ ]:
print(ev_accOnly.options.accretionForceLoss)

In [ ]:
# Evolution of both this code and imripy with just accretion effects but without accretion recoil Force 
sp_spike.m2 = m2
sol_accOnly = evolve(sp_spike, y0, t_end, nSteps=10*nOrbits, gwEmission=False, dynamicalFriction=False, accretion=True, accretionRecoil=False, acc=1e-11)
sol_accOnly = getOrbitalParameters(sp_spike, sol_accOnly)
sp_spike.m2 = m2
ev_accOnly = inspiral.Classic.Evolve(sp_spike, a0, e_0=e0, t_fin = t_end, 
                                     opt=inspiral.Classic.EvolutionOptions(accretion=True, accretionForceLoss=True, accretionRecoilLoss=False,
                                                                           gwEmissionLoss=False, dynamicalFrictionLoss=False, verbose=1, accuracy=1e-11))

In [ ]:
fig, (ax_a, ax_e, ax_m) = plt.subplots(3, 1, figsize=(10, 12))
compareModels(sp_spike, sol_accOnly, ev_accOnly, ax_a, ax_e=ax_e, label="accOnly, w/o accForce")
ax_m.plot(sol_accOnly.t, sol_accOnly.y[4]/m2 - 1.)
ax_m.plot(ev_accOnly.t, ev_accOnly.m2/m2 - 1., linestyle='--')

ax_a.grid(); ax_a.legend()
ax_a.set_yscale('log')
ax_e.grid(); ax_m.grid()

In [ ]:
print(ev_accOnly.options.accretionForceLoss, ev_accOnly.options.accretionRecoilLoss)

In [ ]:
def compareGWsignal(sp, sol, ev, ax_h, ax_hc=None, label=""):
    l1, = ax_h.plot(sol.t, sol.h_plus, label=label+",GR")
    l2, = ax_h.plot(sol.t, ev.h_plus, linestyle='--', label=label+"imripy")
    #ax_h.plot(sol.t, ev.h_cross, linestyle='-.', color=l2.get_c())
    if ax_hc is None:
        return
    ax_hc.loglog(sol.f/ms.hz_to_invpc, 2.*sol.f * np.abs(sol.h_plus_fft), color=l1.get_c())
    ax_hc.loglog(ev.fgw/ms.hz_to_invpc, 2.*ev.fgw * np.abs(ev.h_2_plus),'o', color=l2.get_c(), linestyle=l2.get_linestyle())

In [ ]:
sol_spike = strain(sp_spike, sol_spike)
sol_spike = strainFFT(sp_spike, sol_spike, [1e-4*ms.hz_to_invpc, 1.*ms.hz_to_invpc])

ev_h_plus, ev_h_cross = waveform.h(sp_spike, ev_spike, sol_spike.t)
ev_fgw, ev_h_2_plus, _, __ = waveform.h_n(2,  sp_spike, ev_spike)
ev_spike.h_plus = ev_h_plus; ev_spike.h_cross = ev_h_cross
ev_spike.fgw = ev_fgw; ev_spike.h_2_plus = ev_h_2_plus

In [ ]:
fig, (ax_h, ax_hc) = plt.subplots(2, 1, figsize=(10, 10))
compareGWsignal(sp_spike, sol_spike, ev_spike, ax_h, ax_hc=ax_hc)

ax_h.grid(); ax_h.legend()
ax_h.set_xlim(left = (sol_spike.t[-1]-5/F0), right=sol_spike.t[-1])
ax_hc.grid(); ax_hc.set_xlim(left=F0/2./ms.hz_to_invpc)

### DM model comparison

In [ ]:
# Define different dark matter models
m1= 1e3* ms.solar_mass_to_pc
m2= 1. * ms.solar_mass_to_pc
D = 1e5 

sp_0 = ms.SystemProp(m1, m2, halo.ConstHalo(0.), D)

rho_spike = 226. * ms.solar_mass_to_pc
alpha_spike = 7./3.
r_spike = 0.54
sp_spike = ms.SystemProp(m1, m2, halo.Spike(rho_spike, r_spike, alpha_spike), D)

rho_s = 3.8e-22 * ms.g_cm3_to_invpc2
r_s = 23.1
sp_nfw = ms.SystemProp(m1, m2, halo.NFW(rho_s, r_s), D)

# SIDM profile is missing

In [ ]:
# Define initial conditions
a0 = 50. * sp_0.r_isco()
e0 = 0.
y0 = initial_conditions(sp_0, a0, e0)
print(y0)
F0 = np.sqrt(sp_0.m_total()/a0**3)/ 2./np.pi
nOrbits = 5000
t_end = nOrbits/ F0

In [ ]:
# Evolve systems
sol_0 = evolve(sp_0, y0, t_end, nSteps=10*nOrbits, accretion=False)
sol_spike = evolve(sp_spike, y0, t_end, nSteps=10*nOrbits, accretion=False)
sol_nfw = evolve(sp_nfw, y0, t_end, nSteps=10*nOrbits, accretion=False)

In [ ]:
# Plot evolution of keplerian elements
fig, (ax_a, ax_e) = plt.subplots(2,1, sharex='all', figsize=(10, 8))
fig.subplots_adjust(hspace=0.)
sol_0 = plotOrbitalParameters(sp_0, sol_0, ax_a, ax_e, label='no DM')
sol_spike = plotOrbitalParameters(sp_spike, sol_spike, ax_a, ax_e, label='Spike')
sol_nfw = plotOrbitalParameters(sp_nfw, sol_nfw, ax_a, ax_e, label='nfw')
ax_a.grid(); ax_e.grid()
ax_a.legend()

In [ ]:
# Calculate GW signal
f_band = [1e-4*ms.hz_to_invpc, 1e-1*ms.hz_to_invpc]

sol_0 = strain(sp_0, sol_0)
sol_0 = strainFFT(sp_0, sol_0, f_band)

sol_spike = strain(sp_spike, sol_spike)
sol_spike = strainFFT(sp_spike, sol_spike, f_band)

sol_nfw = strain(sp_nfw, sol_nfw)
sol_nfw = strainFFT(sp_nfw, sol_nfw, f_band)

In [ ]:
# Plot GW signal
fig, (ax_h_plus, ax_f) = plt.subplots(2, 1, figsize=(10,12))
plot_strain(sp_0, sol_0, ax_h_plus, ax_f=ax_f, label="no DM")
plot_strain(sp_spike, sol_spike, ax_h_plus, ax_f=ax_f, label="spike")
plot_strain(sp_nfw, sol_nfw, ax_h_plus, ax_f=ax_f, label="nfw")

ax_h_plus.set_xlabel("t / yr"); ax_h_plus.set_ylabel("$h_+$")
ax_h_plus.grid(); ax_h_plus.legend(); 
#ax_h_plus.set_xlim(left=0., right=10./F0/ms.year_to_pc)
ax_h_plus.set_xlim(left=(sol_spike.t[-1]- 10./F0)/ms.year_to_pc, right=(sol_spike.t[-1])/ms.year_to_pc)
ax_f.set_xlabel("f / Hz"); ax_f.set_ylabel("characteristic strain"); ax_f.grid(); 
print("orbital freq: ", F0/ms.hz_to_invpc)